In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from peft import PeftModel, PeftConfig


class GoralConversation:
    def __init__(
        self,
        message_template=" <s> {role}\n{content} </s>\n",
        system_prompt="Ты — Горал, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.",
        start_token_id=1,
        bot_token_id=9225,
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{"role": "system", "content": system_prompt}]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode(
            [
                self.start_token_id,
            ]
        )
        final_text += " "
        final_text += tokenizer.decode([self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048,
    )
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


# weights_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/rulm2/rulm/self_instruct/models/saiga2_13b_v4"
weights_path = "dim/llama2_13b_dolly_oasst1_chip2"
access_token = ""

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
    token=access_token,
)
model = PeftModel.from_pretrained(
    model,
    weights_path,
    torch_dtype=torch.float16,
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(weights_path)
generation_config = GenerationConfig.from_pretrained(weights_path)
generation_config.do_sample = False


inp = "Напишите интересный пост в блоге о недавней поездке на Гавайи, рассказывая о культурном опыте и достопримечательностях, которые обязательно нужно увидеть."
conversation = GoralConversation(
    start_token_id=1,
    bot_token_id=9225,
)
conversation.add_user_message(inp)
prompt = conversation.get_prompt(tokenizer)

output = generate(model, tokenizer, prompt, generation_config)
print(inp)
print(output)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:45<00:00, 15.11s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Напишите интересный пост в блоге о недавней поездке на Гавайи, рассказывая о культурном опыте и достопримечательностях, которые обязательно нужно увидеть.
Приветствую вас на моем новом блоге! Я только что вернулся из невероятной поездки на Гавайи, и я хочу поделиться своими впечатлениями с вами.

Я прилетел в Гонолулу, столицу штата Гавайи, и сразу был поражен красотой острова. Здесь есть много культурных достопримечательностей, которые вы должны увидеть, если вы посетите.

Важным местом для посещения является Национальный парк Воллеса. Это большой национальный парк, который включает в себя вулканические горы, тропические леса и пляжи. Вы можете совершить поход или прогуляться по тропам, чтобы исследовать его.

Другой популярный туристический объект - пляж Вайкики. Это один из самых известных пляжей в мире, и это отличное место для плавания, серфинга и других водных видов спорта.

Если вы заинтересованы в изучении истории Гавайев, то вы должны посетить Музей Королевства Гавайи. Это муз

### Push to hub


In [2]:
# model.push_to_hub("dim/llama2_13b_dolly_oasst1_chip2")

adapter_model.bin: 100%|██████████| 105M/105M [00:21<00:00, 5.00MB/s]
Upload 1 LFS files: 100%|██████████| 1/1 [00:21<00:00, 21.44s/it]


CommitInfo(commit_url='https://huggingface.co/dim/llama2_13b_dolly_oasst1_chip2/commit/399a0cf9afb8ded90be9bca1f688480e1c05d0ac', commit_message='Upload model', commit_description='', oid='399a0cf9afb8ded90be9bca1f688480e1c05d0ac', pr_url=None, pr_revision=None, pr_num=None)